# Preprocessing the dataset

In [1]:
#https://www.kaggle.com/code/radek1/new-dataset-deberta-v3-large-training을 보고 작성했습니다.

In [2]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
#!pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git

In [3]:
!pip install "/kaggle/input/bitsandbytes-0410/bitsandbytes-0.41.0-py3-none-any.whl"


Processing /kaggle/input/bitsandbytes-0410/bitsandbytes-0.41.0-py3-none-any.whl


In [4]:
!pip install "/kaggle/input/mykagglepippackage/peft-0.4.0-py3-none-any.whl"
#!pip install -U peft --no-index --find-links=file:///kaggle/input/llm-bert/packages/packages/peft

Processing /kaggle/input/mykagglepippackage/peft-0.4.0-py3-none-any.whl


In [5]:

!pip install -U transformers --no-index --find-links=file:///kaggle/input/llm-bert/packages/packages/transformers
#!pip install -U peft --no-index --find-links=file:///kaggle/input/llm-bert/packages/packages/peft
!pip install -U accelerate --no-index --find-links=file:///kaggle/input/llm-bert/packages/packages/accelerate

Looking in links: file:///kaggle/input/llm-bert/packages/packages/transformers
Looking in links: file:///kaggle/input/llm-bert/packages/packages/accelerate


In [6]:
from typing import Optional, Union
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel
model_id = '/kaggle/input/best-llm-starter-pack/nous_hermes/Nous-Hermes-Llama2-13b/snapshots/8f95aa9cd207db7b24179fc779c2b8973e71bee2'
#deberta_v3_large = '/kaggle/input/deberta-v3-large-hf-weights'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

We begin by loading and processing the train data.

In [7]:
df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
df_train = df_train.drop(columns="id")
df_train.shape

(200, 7)

Let's add another 500 examples to the train set!

df_train

In [8]:
#df1 = pd.read_csv("/kaggle/input/wikipedia-stem-1k/stem_1k_v1.csv")
#df1.drop(['id'], axis=1, inplace=True)
#df1.shape

In [9]:
#df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
#df_train = df_train.drop(columns="id")
#df_train.shape

In [10]:
#df_train = pd.concat([
 #    df_train,
#     pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv'),
# ])
#df_train.reset_index(inplace=True, drop=True)
#df_train.shape

In [11]:
df00 = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv')

In [12]:
df00.shape

(6000, 7)

In [13]:
df = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv')
df2=pd.read_csv("/kaggle/input/15k-high-quality-examples/15k_gpt3.5-turbo.csv")
#df2 = df.drop(df.index[:5000])
df2 = df2.reset_index(drop=True)
df_train = pd.concat([
    df_train,
    df,
    df00,df2    
])
  
#df_train = df_train.sample(frac=1)
df_train.reset_index(inplace=True, drop=True)
df_train.shape

(21700, 7)

In [14]:
df_train = df_train.drop_duplicates()
df_train.shape

(21696, 7)

In [15]:
df_train = df2
df_train.shape

(15000, 7)

In [16]:
def create_specific_prompt(pt):
    prompt = ("As a science expert solving a science exam question, you are tasked to answer the following."
              "If it is correct, explain why you judged it as correct, and if it is wrong, think of the reason why you judged it as wrong. Let’s work this out step by step to be sure we have the right answer."
              "Here's the question:\n\n")
    return prompt+pt
              

In [17]:
df_train['prompt']=df_train['prompt'].apply(create_specific_prompt)

In [18]:
df_train['prompt'][0]

"As a science expert solving a science exam question, you are tasked to answer the following.If it is correct, explain why you judged it as correct, and if it is wrong, think of the reason why you judged it as wrong. Let’s work this out step by step to be sure we have the right answer.Here's the question:\n\nWho was responsible for the reorganisation of New Army reserve units during the First World War?"

Now that we have gone from 200 -> 700 train examples, let us preprocess the data and begin training.

In [19]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [example['prompt']] * 5
    second_sentences = [example[option] for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [20]:
deberta_v3_large = '/kaggle/input/2023kagglellm-deberta-v3-large-model1'
model = deberta_v3_large
model_id = model

We first create a HuggingFace `Dataset`.

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model)
dataset = Dataset.from_pandas(df_train)
dataset

Dataset({
    features: ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'],
    num_rows: 15000
})

In [22]:
#tokenizer = AutoTokenizer.from_pretrained(
  #  model,
#    use_fast=False,
 #   trust_remote_code=True,
  #  padding_side="left",
  #  pad_token="<|endoftext|>"
#)#

#dataset = Dataset.from_pandas(df_train)
#dataset

And let us now preprocess the examples for training.

In [23]:
tokenized_dataset = dataset.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_dataset

  0%|          | 0/15000 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 15000
})

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = '/kaggle/input/deberta-v3-large-hf-weights' 
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
model = AutoModelForMultipleChoice.from_pretrained(model_id,quantization_config=bnb_config)

Some weights of the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights were not used when initializing DebertaV2ForMultipleChoice: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

In [26]:
#model = AutoModelForMultipleChoice.from_pretrained(deberta_v3_large)


In [27]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [28]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [29]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r= 64,
    lora_alpha=32,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 19005440 || all params: 302548993 || trainable%: 6.281772684664001


In [30]:
# model = PeftModel.from_pretrained(model, lora_model_dir)

# Training

In [31]:
training_args = TrainingArguments(
    warmup_ratio=0.8,
    learning_rate=5e-6,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2.5,
    report_to='none',
    output_dir='.',
    optim='adamw_torch'

)

# model = AutoModelForMultipleChoice.from_pretrained(deberta_v3_large,quantization_config=bnb_config)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_dataset
)

trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.613800
1000,1.615500
1500,1.614200
2000,1.617700
2500,1.608500
3000,1.613000
3500,1.609300
4000,1.614100
4500,1.613200
5000,1.614300


TrainOutput(global_step=18750, training_loss=1.5555414664713543, metrics={'train_runtime': 12399.9745, 'train_samples_per_second': 3.024, 'train_steps_per_second': 1.512, 'total_flos': 2.25969141977514e+16, 'train_loss': 1.5555414664713543, 'epoch': 2.5})

# Predicting on the test set

Now that we have trained our model, let us predict on the test set.

In [32]:
import pandas as pd
from datasets import Dataset

In [33]:
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
test_df['answer'] = 'A' # dummy answer that allows us to preprocess the test dataset just like we preprocessed the train dataset
test_df['prompt'] = test_df['prompt'].apply(create_specific_prompt)
tokenized_test_dataset = Dataset.from_pandas(test_df.drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E'])

  0%|          | 0/200 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
test_df

,id,prompt,A,B,C,D,E,answer
0,0,As a science expert solving a science exam que...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,A
1,1,As a science expert solving a science exam que...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,As a science expert solving a science exam que...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,As a science expert solving a science exam que...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,A
4,4,As a science expert solving a science exam que...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,A
...,...,...,...,...,...,...,...,...
195,195,As a science expert solving a science exam que...,The three moment theorem expresses the relatio...,The three moment theorem is used to calculate ...,The three moment theorem describes the relatio...,The three moment theorem is used to calculate ...,The three moment theorem is used to derive the...,A
196,196,As a science expert solving a science exam que...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,The throttling process is a steady adiabatic f...,The throttling process is a steady flow of a f...,The throttling process is a steady adiabatic f...,A
197,197,As a science expert solving a science exam que...,"The excess base metal will often solidify, bec...",The excess base metal will often crystallize-o...,"The excess base metal will often dissolve, bec...","The excess base metal will often liquefy, beco...","The excess base metal will often evaporate, be...",A
198,198,As a science expert solving a science exam que...,Mass is a property that determines the weight ...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is an inertial property that determines a...,Mass is a property that determines the size of...,A


In [35]:
test_predictions = trainer.predict(tokenized_test_dataset).predictions
test_predictions[:4]

array([[-0.10422602,  1.4582524 ,  0.01658112,  1.6056869 , -0.21981567],
       [-0.51512194, -0.9311666 , -1.5219071 , -1.4329563 , -1.3814052 ],
       [ 1.2669116 ,  0.44653463, -0.20179509,  1.5198768 ,  0.8809513 ],
       [ 1.2540419 , -0.5024787 ,  1.0226139 , -0.04397823, -0.05822211]],
      dtype=float32)

The predictions are values output from the last layer of our neural network.

Let's obtain the predicted answer ids by sorting them from largest to the smallest.

In [36]:
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_ids[:3]

array([[3, 1, 2, 0, 4],
       [0, 1, 4, 3, 2],
       [3, 0, 4, 1, 2]])

Let us now assign a letter corresponding to each predicted id (0 -> 'A', 1 -> 'B', etc). 

In [37]:
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
predictions_as_answer_letters[:3]

array([['D', 'B', 'C', 'A', 'E'],
       ['A', 'B', 'E', 'D', 'C'],
       ['D', 'A', 'E', 'B', 'C']], dtype='<U1')

In [38]:
predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]
predictions_as_string[:3]

['D B C', 'A B E', 'D A E']

In [39]:
submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv').head()

,id,prediction
0,0,D B C
1,1,A B E
2,2,D A E
3,3,A C D
4,4,D A C
